In [2]:
#importing libraries
import numpy as np
import pandas as pd
import csv

#importing dataset (book)
books=pd.read_csv("./BX-Books.csv",sep=';',error_bad_lines=False,encoding='latin-1',skip_blank_lines=True)

#selection of required column

#books=books[['ISBN','Book-Title','Book-Author']]
books=books.iloc[:,0:3]
books.columns=["ISBN","Book Name","Author"]

#importing another dataset
ratings=pd.read_csv("./BX-Book-Ratings.csv",sep=';',error_bad_lines=False,warn_bad_lines=Fal,encoding='latin-1')

#selection of required column
ratings.columns=["User_id","ISBN","Rating"]

#selecting those user who read min 200 books
user_min=ratings['User_id'].value_counts()
user_min=user_min[user_min>200]
ratings=ratings[ratings['User_id'].isin(user_min.index)]

#here we select ratings from those users which is in user_min so for that we choose their index(i.e,user_id)
ratings.head(1)
ratings_with_bookname=ratings.merge(books, on='ISBN')

#here value reduce becoz there are books which are not in book
final_ratings_with_bookname=ratings_with_bookname.groupby('Book Name')['Rating'].agg('count').reset_index().rename(columns={'Rating':'num_ratings'})


#merging two data
ultimate_final_ratings_with_bookname=ratings_with_bookname.merge(final_ratings_with_bookname, on="Book Name")
#masking operation
mask=ultimate_final_ratings_with_bookname["num_ratings"]>=50
ultimate_final_ratings_with_bookname=ultimate_final_ratings_with_bookname[mask]
ultimate_final=ultimate_final_ratings_with_bookname.drop_duplicates(['User_id','Book Name'])
ultimate_pivot=ultimate_final.pivot_table(values='Rating',index='Book Name',columns='User_id').fillna(0)

#sparse matrix
from scipy.sparse import csr_matrix
ultimate_sparse=csr_matrix(ultimate_pivot)

#applying NN algorithm
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(metric='cosine',algorithm='brute')
nn.fit(ultimate_sparse)
#distances,suggestion=nn.kneighbors(ultimate_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)
#suggestion=suggestion.ravel()

#function block  of book recomender
def suggest(book_name):
   #taking book index 
    bindex=np.where(ultimate_pivot.index==book_name)[0][0]
    #
    distances,suggestion=nn.kneighbors(ultimate_pivot.iloc[bindex,:].values.reshape(1,-1),n_neighbors=6)
    #flattenning of matrix
    suggestion=suggestion.ravel()
    for i in range(len(suggestion)):
        if i==0:
            print("book suggestion for:{}\n".format(ultimate_pivot.index[suggestion[i]]),flush=True)
        else:
             print(ultimate_pivot.index[suggestion[i]])
suggest(input("enter book name"))

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.

enter book name1984
book suggestion for:1984

Animal Farm
The Handmaid's Tale
The Catcher in the Rye
Lord of the Flies
The Vampire Lestat (Vampire Chronicles, Book II)
